<span style="color: Gold"> KoBERT
- https://huggingface.co/
- https://huggingface.co/skt/kobert-base-v1 --> Kobert 중에서 skt에서 만든게 가장 유명함
- https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf --> 사용방법 기재되어있음


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
!pip install sentencepiece

In [ ]:
# https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf
# 상기 사이트에서 Tokenizer (BPE-dropout) 부분 복사 사용
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")
# 상기 인코더 출력결과 비교 [2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [ ]:
tokenizer("한국어 모델을 공유합니다.")  # 기존과 동일한 출력결과 나옴 (input_ids, token_type_ids, attention_mask)

{'input_ids': [2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf
# 상기 사이트에서 Model 부분 복사 사용
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),  #torch.tensor --> 리스트를 텐서화시킨것
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape
# 출력결과 torch.Size([1, 768])
# 임베딩 차원(768)은 모델 자체에 들어가있으므로 변경불가

torch.Size([1, 768])

In [ ]:
# 데이터셋 클래스
import torch
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        # BERT encoding 되면 input_ids, token_type_ids, attention_mask --> 딕셔너리 --> torch.tensor.. 왜?
        item = {key : torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

In [ ]:
# 한국어 영화리뷰 데이터
# 데이터 로드 및 분할 (여기서는 성능상 일부 데이터만 사용)
import pandas as pd
from sklearn.model_selection import train_test_split
url = "https://drive.google.com/uc?id=1KOKgZ4qCg49bgj1QNTwk1Vd29soeB27o"
df = pd.read_csv(url)
print(len(df))
df = df.sample(frac=0.1)  #원래 데이터 중에서 frac% 만큼 가지고 옴
print(len(df))

X = df.review.tolist() #list로 변환
y = (df.rating >= 6).values.astype(int)  # (df.rating >= 6) rating 6이상이면 True가되고, .values.astype(int)에 의해 True는1 , False 0 으로 변환

X_, X_test, y_, y_test = train_test_split(X, y, stratify=y,  test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, stratify=y_, test_size=0.2, random_state=42)

14725
1472


In [ ]:
from torch.utils.data import DataLoader
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

# 토큰화
train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
val_input = tokenizer(X_val, truncation=True, padding=True, max_length=512, return_tensors="pt")
test_input = tokenizer(X_test, truncation=True, padding=True, max_length=512, return_tensors="pt")


# DataSet 생성
train_dataset = OurDataset(train_input, y_train)
val_dataset = OurDataset(val_input, y_val)
test_dataset = OurDataset(test_input, y_test)


# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=8) # 기존에는 문장갯수가 batch_size였는데.. 여기서는???
val_loader = DataLoader(val_dataset, batch_size=16) # batch_size 달리 줘도 됨..왜?
test_loader = DataLoader(test_dataset, batch_size=16)


# KoBERT  한국어 전용 모델 로드
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')


# BERT를 포함한 신경망 모델 만들기
import torch.nn as nn
class MyModel(nn.Module):  #전용모델에 MyModel을 붙임..???
    def __init__(self, pretrained_model, token_size, num_labels):
        super(MyModel, self).__init__()
        self.pretrained_model = pretrained_model
        self.token_size = token_size
        self.num_labels = num_labels
        # 분류기 정의
        self.clf = nn.Linear(self.token_size, self.num_labels)

    # 조립 forward - 모델완성
    def forward(self, inputs):
        outputs = self.pretrained_model(**inputs)  #**inputs:key, value #ouputs = [batch, embeding_dim, num_labels]
        bert_clf_token = outputs.last_hidden_state[:,0,:] #모든 배치크기에 0번째 clf에 데이터가 다 들어가있으므로 가져옴
        return self.clf(bert_clf_token)

model = MyModel(model, token_size=model.config.hidden_size, num_labels=2) #(model.token_size=model.config.hidden_size,??



# 학습 - 미니 batch
    # device
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

    # optimize
from torch.optim import AdamW
optim = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

    # 학습 스케쥴러
from transformers import get_linear_schedule_with_warmup  #학습속도를 점점 줄여나감
scheduler = get_linear_schedule_with_warmup (optimizer=optim, num_warmup_steps=200, num_training_steps=len(train_loader))


import numpy as np
    # epoch 수만큼 loop돌면서 학습
for epoch in range(3):
    total_loss = 0
    for batch in train_loader :
        model.train()
        optim.zero_grad()
            # 배치에서는 label을 제외하고 입력만 추출
        inputs = {k : v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(inputs)  #MyModel에서 **inputs 정의되어 있으니, 여기서는 그냥 inputs로만 기재
        loss = criterion(outputs, labels)
        loss.backward() #gradient 계산
        optim.step()  #gradient 적용
        scheduler.step()
        total_loss += loss.item()
    print(f'epoch : {epoch+1}, train loss : {total_loss/len(train_loader)}')

    # val 데이터로 해당 epoch 학습된 모델을 평가
    with torch.no_grad():
        model.eval()
        total_val_loss = 0
        for batch in val_loader :
            inputs = {k : v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
    print(f'epoch : {epoch+1}, validation loss : {total_val_loss/len(val_loader)}')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipython-input-1693230857.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key : torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}


epoch : 1, train loss : 0.5819743244072139
epoch : 1, validation loss : 0.5272265096505483
epoch : 2, train loss : 0.4972589051319381
epoch : 2, validation loss : 0.4612987399101257
epoch : 3, train loss : 0.4282532634876542
epoch : 3, validation loss : 0.4612987399101257


In [ ]:
!pip install evaluate

In [ ]:
# 평가
import evaluate
model.eval()
with torch.no_grad():
    for batch in test_loader :
        inputs = {k : v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=-1)
        metrics = evaluate.load("accuracy")
        metrics.add_batch(predictions=predictions, references=labels)
metrics.compute()

/tmp/ipython-input-1693230857.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key : torch.tensor(val[idx].clone().detach()) for key, val in self.encodings.items()}


{'accuracy': 0.8571428571428571}

In [ ]:
# 추론
df2 = df.reset_index(drop=True)
df2[-5:].review.tolist()

['역대 마블 최고',
 '천만레벨',
 '볼거리는 풍성하나 문제가 되는 장면들.... 1,2 주요한 테마부분을 실랄하게 정제시키지못하고 슬쩍 삽화화한 것은 아쉽다. 아쉽기는 차태연의 연기도 마찬가지..... 외모가 너무 윤택혀^^ 배우가 진정연기자가 되려면 예능은 안해야 되는 것 같다. 뭐 배우보다 연예인이라면 할말 없지만..',
 '아무도 몰랐던 광주이야기에 감동',
 '재밌다.']

In [ ]:
import torch
inputs = tokenizer(df2[-5:].review.values.tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512)
inputs = {k : v.to(device) for k, v in inputs.items()}
outputs = model(inputs)
preds = torch.softmax(outputs, dim=-1)
preds = torch.argmax(preds, dim=-1)
print(f'예측 결과 1 긍정 / 0 부정 == {preds}')

예측 결과 1 긍정 / 0 부정 == tensor([1, 1, 1, 1, 1], device='cuda:0')
